# Weather Mispricing Detector

Fetch live London weather from **4 sources**, compute fair values for WX_SPOT & WX_SUM, and detect mispricings.

**Sources:**
1. Open-Meteo (settlement source — no key needed)
2. OpenWeatherMap (API key)
3. WeatherAPI.com (API key)
4. MET Office DataHub (free, no key for basic obs)

**Markets:**
- **WX_SPOT** = `round(Temp_F) × Humidity` at Sunday 12:00
- **WX_SUM** = `Σ (Temp_F × Humidity) / 100` over all 15-min intervals (Sat 12:00 → Sun 12:00)

In [ ]:
!pip install requests pandas numpy -q

In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import time
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# CONFIG
# ============================================================
LONDON_LAT, LONDON_LON = 51.5074, -0.1278

OWM_API_KEY = "ad6b547d314dbfae707b72c101dbc740"      # OpenWeatherMap
WAPI_API_KEY = "c5031f71c3874ac69e0191831262802"       # WeatherAPI.com

def c_to_f(c):
    """Celsius to Fahrenheit"""
    return c * 9/5 + 32

print("Ready.")

---
## Source 1: Open-Meteo (THE settlement source)

This is the most important source — settlement uses Open-Meteo data.
- 15-min resolution
- Past observations + forecast
- No API key needed

In [ ]:
def fetch_open_meteo(past_steps=96, forecast_steps=96):
    """
    Fetch 15-min weather from Open-Meteo.
    96 steps = 24 hours.
    Returns DataFrame with time, temp_c, temp_f, humidity, temp_f_x_humidity.
    """
    variables = "temperature_2m,relative_humidity_2m"
    resp = requests.get("https://api.open-meteo.com/v1/forecast", params={
        "latitude": LONDON_LAT, "longitude": LONDON_LON,
        "minutely_15": variables,
        "past_minutely_15": past_steps,
        "forecast_minutely_15": forecast_steps,
        "timezone": "Europe/London",
    })
    resp.raise_for_status()
    m = resp.json()["minutely_15"]
    
    df = pd.DataFrame({
        "time": pd.to_datetime(m["time"]),
        "temp_c": m["temperature_2m"],
        "humidity": m["relative_humidity_2m"],
    })
    df["temp_f"] = c_to_f(df["temp_c"])
    df["temp_f_rounded"] = df["temp_f"].round()
    df["t_x_h"] = df["temp_f_rounded"] * df["humidity"]
    df["source"] = "open_meteo"
    return df

df_om = fetch_open_meteo()
print(f"Open-Meteo: {len(df_om)} readings, {df_om.time.min()} → {df_om.time.max()}")
df_om.tail(5)

---
## Source 2: OpenWeatherMap

Current conditions + 5-day/3-hour forecast.
Useful as a second opinion to cross-validate Open-Meteo.

In [ ]:
def fetch_openweathermap_current():
    """
    Fetch current weather from OpenWeatherMap.
    Returns dict with temp_c, temp_f, humidity, timestamp.
    """
    resp = requests.get("https://api.openweathermap.org/data/2.5/weather", params={
        "lat": LONDON_LAT, "lon": LONDON_LON,
        "appid": OWM_API_KEY,
        "units": "metric",  # Celsius
    })
    resp.raise_for_status()
    d = resp.json()
    
    temp_c = d["main"]["temp"]
    humidity = d["main"]["humidity"]
    return {
        "time": pd.Timestamp.fromtimestamp(d["dt"]),
        "temp_c": temp_c,
        "temp_f": c_to_f(temp_c),
        "temp_f_rounded": round(c_to_f(temp_c)),
        "humidity": humidity,
        "t_x_h": round(c_to_f(temp_c)) * humidity,
        "source": "owm",
    }

def fetch_openweathermap_forecast():
    """
    Fetch 5-day/3-hour forecast from OpenWeatherMap.
    Returns DataFrame.
    """
    resp = requests.get("https://api.openweathermap.org/data/2.5/forecast", params={
        "lat": LONDON_LAT, "lon": LONDON_LON,
        "appid": OWM_API_KEY,
        "units": "metric",
    })
    resp.raise_for_status()
    data = resp.json()
    
    rows = []
    for item in data["list"]:
        temp_c = item["main"]["temp"]
        humidity = item["main"]["humidity"]
        rows.append({
            "time": pd.Timestamp.fromtimestamp(item["dt"]),
            "temp_c": temp_c,
            "temp_f": c_to_f(temp_c),
            "temp_f_rounded": round(c_to_f(temp_c)),
            "humidity": humidity,
            "t_x_h": round(c_to_f(temp_c)) * humidity,
            "source": "owm_forecast",
        })
    return pd.DataFrame(rows)

# Current
owm_current = fetch_openweathermap_current()
print("=== OWM Current ===")
print(f"  Temp: {owm_current['temp_c']:.1f}°C / {owm_current['temp_f']:.1f}°F")
print(f"  Humidity: {owm_current['humidity']}%")
print(f"  T×H: {owm_current['t_x_h']}")

# Forecast
df_owm_fc = fetch_openweathermap_forecast()
print(f"\nOWM Forecast: {len(df_owm_fc)} readings (3-hourly)")
df_owm_fc.head(5)

---
## Source 3: WeatherAPI.com

Real-time + forecast. Gives both current conditions and hourly forecasts.

In [ ]:
def fetch_weatherapi_current():
    """
    Fetch current weather from WeatherAPI.com.
    """
    resp = requests.get("https://api.weatherapi.com/v1/current.json", params={
        "key": WAPI_API_KEY,
        "q": f"{LONDON_LAT},{LONDON_LON}",
    })
    resp.raise_for_status()
    d = resp.json()["current"]
    
    return {
        "time": pd.Timestamp(d["last_updated"]),
        "temp_c": d["temp_c"],
        "temp_f": d["temp_f"],
        "temp_f_rounded": round(d["temp_f"]),
        "humidity": d["humidity"],
        "t_x_h": round(d["temp_f"]) * d["humidity"],
        "source": "weatherapi",
    }

def fetch_weatherapi_forecast(days=2):
    """
    Fetch hourly forecast from WeatherAPI.com.
    """
    resp = requests.get("https://api.weatherapi.com/v1/forecast.json", params={
        "key": WAPI_API_KEY,
        "q": f"{LONDON_LAT},{LONDON_LON}",
        "days": days,
    })
    resp.raise_for_status()
    data = resp.json()
    
    rows = []
    for day in data["forecast"]["forecastday"]:
        for hour in day["hour"]:
            rows.append({
                "time": pd.Timestamp(hour["time"]),
                "temp_c": hour["temp_c"],
                "temp_f": hour["temp_f"],
                "temp_f_rounded": round(hour["temp_f"]),
                "humidity": hour["humidity"],
                "t_x_h": round(hour["temp_f"]) * hour["humidity"],
                "source": "weatherapi_forecast",
            })
    return pd.DataFrame(rows)

# Current
wapi_current = fetch_weatherapi_current()
print("=== WeatherAPI Current ===")
print(f"  Temp: {wapi_current['temp_c']:.1f}°C / {wapi_current['temp_f']:.1f}°F")
print(f"  Humidity: {wapi_current['humidity']}%")
print(f"  T×H: {wapi_current['t_x_h']}")

# Forecast
df_wapi_fc = fetch_weatherapi_forecast()
print(f"\nWeatherAPI Forecast: {len(df_wapi_fc)} readings (hourly)")
df_wapi_fc.head(5)

---
## Source 4: MET Office (via DataPoint — free, needs registration)

The MET Office DataPoint API requires a free API key (register at https://datahub.metoffice.gov.uk).

**Alternative**: We scrape the MET Office's free observation data for London (no key needed), or we use their public site data.

Since getting a MET key takes time, we'll use the **wttr.in** service as a lightweight 4th source — it aggregates MET-style data and is free with no key.

In [ ]:
def fetch_wttr_current():
    """
    Fetch current weather from wttr.in (aggregates multiple sources).
    No API key needed.
    """
    resp = requests.get("https://wttr.in/London", params={"format": "j1"})
    resp.raise_for_status()
    d = resp.json()["current_condition"][0]
    
    temp_c = float(d["temp_C"])
    temp_f = float(d["temp_F"])
    humidity = int(d["humidity"])
    
    return {
        "time": pd.Timestamp.now(),
        "temp_c": temp_c,
        "temp_f": temp_f,
        "temp_f_rounded": round(temp_f),
        "humidity": humidity,
        "t_x_h": round(temp_f) * humidity,
        "source": "wttr_in",
    }

wttr_current = fetch_wttr_current()
print("=== wttr.in Current ===")
print(f"  Temp: {wttr_current['temp_c']:.1f}°C / {wttr_current['temp_f']:.1f}°F")
print(f"  Humidity: {wttr_current['humidity']}%")
print(f"  T×H: {wttr_current['t_x_h']}")

---
## Compare All Sources — Current Snapshot

This is the key table: see how sources diverge right now.

In [ ]:
def fetch_all_current():
    """
    Fetch current readings from all 4 sources.
    Returns DataFrame for easy comparison.
    """
    results = []
    
    # 1. Open-Meteo — get the latest observation
    try:
        df_om = fetch_open_meteo(past_steps=4, forecast_steps=4)
        # Most recent past observation
        now = pd.Timestamp.now()
        past = df_om[df_om.time <= now.strftime("%Y-%m-%d %H:%M")]
        if len(past) > 0:
            latest = past.iloc[-1]
            results.append({
                "source": "Open-Meteo ⭐",
                "temp_c": latest["temp_c"],
                "temp_f": latest["temp_f"],
                "temp_f_rounded": latest["temp_f_rounded"],
                "humidity": latest["humidity"],
                "t_x_h": latest["t_x_h"],
                "time": latest["time"],
            })
    except Exception as e:
        print(f"Open-Meteo error: {e}")
    
    # 2. OpenWeatherMap
    try:
        owm = fetch_openweathermap_current()
        owm["source"] = "OpenWeatherMap"
        results.append(owm)
    except Exception as e:
        print(f"OWM error: {e}")
    
    # 3. WeatherAPI.com
    try:
        wapi = fetch_weatherapi_current()
        wapi["source"] = "WeatherAPI.com"
        results.append(wapi)
    except Exception as e:
        print(f"WeatherAPI error: {e}")
    
    # 4. wttr.in
    try:
        wttr = fetch_wttr_current()
        wttr["source"] = "wttr.in"
        results.append(wttr)
    except Exception as e:
        print(f"wttr.in error: {e}")
    
    df = pd.DataFrame(results)
    return df

df_comparison = fetch_all_current()
print("\n" + "="*70)
print("    CURRENT WEATHER — ALL SOURCES COMPARISON")
print("="*70)
print(df_comparison[["source", "temp_c", "temp_f_rounded", "humidity", "t_x_h"]].to_string(index=False))

# Stats
print(f"\n--- Consensus ---")
print(f"  Temp F (mean):   {df_comparison['temp_f'].mean():.1f}°F")
print(f"  Temp F (std):    {df_comparison['temp_f'].std():.2f}°F")
print(f"  Humidity (mean): {df_comparison['humidity'].mean():.1f}%")
print(f"  Humidity (std):  {df_comparison['humidity'].std():.2f}%")
print(f"  T×H (mean):     {df_comparison['t_x_h'].mean():.0f}")
print(f"  T×H (std):      {df_comparison['t_x_h'].std():.0f}")
print(f"  T×H (range):    {df_comparison['t_x_h'].min():.0f} — {df_comparison['t_x_h'].max():.0f}")

---
## Fair Value Calculator

### WX_SPOT Fair Value
We weight Open-Meteo higher (it's the settlement source) and use others to adjust.

### WX_SUM Fair Value
`fair_value = observed_sum + forecast_remaining_sum`

As time passes, observed_sum grows (known exactly), forecast_remaining shrinks (less uncertainty).

In [ ]:
# ============================================================
# SESSION CONFIG — adjust these for your trading session
# ============================================================
# The session runs Saturday 12:00 → Sunday 12:00 London time
# Adjust these dates for your actual session:
SESSION_START = pd.Timestamp("2026-03-07 12:00", tz="Europe/London")
SESSION_END   = pd.Timestamp("2026-03-08 12:00", tz="Europe/London")
TOTAL_INTERVALS = 96  # 24h / 15min = 96 intervals

print(f"Session: {SESSION_START} → {SESSION_END}")
print(f"Total 15-min intervals: {TOTAL_INTERVALS}")

In [ ]:
def compute_wx_spot_fair_value():
    """
    Estimate WX_SPOT = round(Temp_F) × Humidity at settlement (Sunday 12:00).
    
    Strategy: 
    - Use Open-Meteo forecast for Sunday 12:00 as primary estimate
    - Cross-validate with OWM and WeatherAPI forecasts
    - Weight: Open-Meteo 50%, OWM 25%, WeatherAPI 25%
    """
    estimates = []
    weights = []
    
    # 1. Open-Meteo forecast — HIGHEST WEIGHT (settlement source)
    try:
        df_om = fetch_open_meteo(past_steps=4, forecast_steps=96)
        # Find the forecast closest to settlement time
        settlement_str = SESSION_END.strftime("%Y-%m-%d %H:%M")
        closest = df_om.iloc[(df_om.time - pd.Timestamp(settlement_str)).abs().argsort().iloc[0]]
        om_estimate = closest["t_x_h"]
        estimates.append(om_estimate)
        weights.append(0.50)
        print(f"  Open-Meteo forecast for settlement: T_F={closest['temp_f']:.1f}, H={closest['humidity']}, T×H={om_estimate:.0f}")
    except Exception as e:
        print(f"  Open-Meteo error: {e}")
    
    # 2. OWM forecast (3-hourly) — find closest to settlement
    try:
        df_owm = fetch_openweathermap_forecast()
        settlement_ts = SESSION_END.tz_localize(None) if SESSION_END.tzinfo else SESSION_END
        closest_owm = df_owm.iloc[(df_owm.time - pd.Timestamp(settlement_str)).abs().argsort().iloc[0]]
        owm_estimate = closest_owm["t_x_h"]
        estimates.append(owm_estimate)
        weights.append(0.25)
        print(f"  OWM forecast for settlement:        T_F={closest_owm['temp_f']:.1f}, H={closest_owm['humidity']}, T×H={owm_estimate:.0f}")
    except Exception as e:
        print(f"  OWM error: {e}")
    
    # 3. WeatherAPI forecast (hourly)
    try:
        df_wapi = fetch_weatherapi_forecast(days=2)
        closest_wapi = df_wapi.iloc[(df_wapi.time - pd.Timestamp(settlement_str)).abs().argsort().iloc[0]]
        wapi_estimate = closest_wapi["t_x_h"]
        estimates.append(wapi_estimate)
        weights.append(0.25)
        print(f"  WeatherAPI forecast for settlement: T_F={closest_wapi['temp_f']:.1f}, H={closest_wapi['humidity']}, T×H={wapi_estimate:.0f}")
    except Exception as e:
        print(f"  WeatherAPI error: {e}")
    
    if not estimates:
        print("  ERROR: No sources available!")
        return None
    
    # Weighted average
    weights = np.array(weights)
    weights = weights / weights.sum()  # normalize
    fair_value = np.average(estimates, weights=weights)
    
    print(f"\n  >>> WX_SPOT Fair Value: {fair_value:.0f}")
    print(f"  >>> Range: {min(estimates):.0f} — {max(estimates):.0f}")
    return fair_value

print("=" * 60)
print("  WX_SPOT FAIR VALUE ESTIMATE")
print("=" * 60)
spot_fv = compute_wx_spot_fair_value()

In [ ]:
def compute_wx_sum_fair_value():
    """
    Estimate WX_SUM = Σ (Temp_F_rounded × Humidity) / 100 over all 15-min intervals.
    
    Strategy:
    - For past intervals: use Open-Meteo observations (exact)
    - For future intervals: use Open-Meteo forecast (estimated)
    - Cross-validate future part with OWM/WeatherAPI
    """
    # Get full Open-Meteo data covering the session
    df_om = fetch_open_meteo(past_steps=96, forecast_steps=96)
    
    session_start_str = SESSION_START.strftime("%Y-%m-%d %H:%M")
    session_end_str = SESSION_END.strftime("%Y-%m-%d %H:%M")
    
    # Filter to session window
    df_session = df_om[
        (df_om.time >= session_start_str) & 
        (df_om.time <= session_end_str)
    ].copy()
    
    if len(df_session) == 0:
        print("  WARNING: No data in session window. Check SESSION_START/END dates.")
        print(f"  Available data: {df_om.time.min()} → {df_om.time.max()}")
        return None
    
    # Split into observed (past) vs forecast (future)
    now_str = pd.Timestamp.now().strftime("%Y-%m-%d %H:%M")
    df_observed = df_session[df_session.time <= now_str].copy()
    df_forecast = df_session[df_session.time > now_str].copy()
    
    # Compute sums
    observed_sum = (df_observed["t_x_h"] / 100).sum()
    forecast_sum = (df_forecast["t_x_h"] / 100).sum()
    total_estimate = observed_sum + forecast_sum
    
    pct_observed = len(df_observed) / TOTAL_INTERVALS * 100
    
    print(f"  Intervals observed: {len(df_observed)}/{TOTAL_INTERVALS} ({pct_observed:.0f}%)")
    print(f"  Intervals remaining: {len(df_forecast)}/{TOTAL_INTERVALS}")
    print(f"")
    print(f"  Observed sum (exact):     {observed_sum:.1f}")
    print(f"  Forecast sum (estimated): {forecast_sum:.1f}")
    print(f"")
    print(f"  >>> WX_SUM Fair Value: {total_estimate:.0f}")
    print(f"  >>> Uncertainty: ±{forecast_sum * 0.1:.0f} (rough 10% on forecast part)")
    
    return {
        "fair_value": total_estimate,
        "observed_sum": observed_sum,
        "forecast_sum": forecast_sum,
        "pct_observed": pct_observed,
        "n_observed": len(df_observed),
        "n_remaining": len(df_forecast),
    }

print("=" * 60)
print("  WX_SUM FAIR VALUE ESTIMATE")
print("=" * 60)
sum_fv = compute_wx_sum_fair_value()

---
## Mispricing Detector

Compare our fair values against the current market price.
You can manually enter the market mid or plug this into the exchange API.

In [ ]:
def detect_mispricing(fair_value, market_price, product_name, threshold_pct=2.0):
    """
    Compare fair value vs market and decide action.
    
    Args:
        fair_value: our estimated fair value
        market_price: current market mid-price
        product_name: e.g. "WX_SPOT" or "WX_SUM"
        threshold_pct: minimum % difference to act on
    """
    if fair_value is None or market_price is None:
        print(f"  {product_name}: Cannot compute — missing data")
        return None
    
    diff = fair_value - market_price
    diff_pct = (diff / market_price) * 100
    
    print(f"\n  {product_name}:")
    print(f"    Fair Value:   {fair_value:.0f}")
    print(f"    Market Price: {market_price:.0f}")
    print(f"    Difference:   {diff:+.0f} ({diff_pct:+.1f}%)")
    
    if abs(diff_pct) < threshold_pct:
        print(f"    → NO TRADE (within {threshold_pct}% threshold)")
        return 0
    elif diff > 0:
        print(f"    → 🟢 BUY SIGNAL — market is cheap by {diff_pct:.1f}%")
        return 1
    else:
        print(f"    → 🔴 SELL SIGNAL — market is expensive by {abs(diff_pct):.1f}%")
        return -1

# ============================================================
# ENTER MARKET PRICES HERE (from the exchange orderbook)
# ============================================================
MARKET_WX_SPOT = None  # e.g. 950
MARKET_WX_SUM  = None  # e.g. 4500

print("=" * 60)
print("  MISPRICING DETECTION")
print("=" * 60)

if MARKET_WX_SPOT is not None and spot_fv is not None:
    detect_mispricing(spot_fv, MARKET_WX_SPOT, "WX_SPOT")
else:
    print("\n  WX_SPOT: Set MARKET_WX_SPOT to the current market mid-price")

if MARKET_WX_SUM is not None and sum_fv is not None:
    detect_mispricing(sum_fv["fair_value"], MARKET_WX_SUM, "WX_SUM")
else:
    print("  WX_SUM:  Set MARKET_WX_SUM to the current market mid-price")

---
## Live Monitoring Loop

Run this cell to continuously fetch data, compute fair values, and print updates.
Interrupt the kernel to stop.

In [ ]:
def live_monitor(interval_seconds=60, n_iterations=None):
    """
    Continuously fetch all sources, compute fair values, and log.
    
    Args:
        interval_seconds: how often to poll (default 60s)
        n_iterations: stop after N iterations (None = infinite)
    """
    history = []  # track fair value over time
    i = 0
    
    while True:
        i += 1
        if n_iterations and i > n_iterations:
            break
            
        now = datetime.now()
        print(f"\n{'='*60}")
        print(f"  [{now.strftime('%H:%M:%S')}] Iteration {i}")
        print(f"{'='*60}")
        
        # Fetch all current readings
        try:
            df_all = fetch_all_current()
            print("\n  Current readings:")
            print(df_all[["source", "temp_f_rounded", "humidity", "t_x_h"]].to_string(index=False))
            
            mean_txh = df_all["t_x_h"].mean()
            std_txh = df_all["t_x_h"].std()
            print(f"\n  Consensus T×H: {mean_txh:.0f} ± {std_txh:.0f}")
            
            history.append({
                "time": now,
                "mean_txh": mean_txh,
                "std_txh": std_txh,
                "n_sources": len(df_all),
            })
        except Exception as e:
            print(f"  Error fetching: {e}")
        
        if n_iterations and i >= n_iterations:
            break
        
        print(f"\n  Sleeping {interval_seconds}s...")
        time.sleep(interval_seconds)
    
    return pd.DataFrame(history)

# Run 3 iterations as a test (60s apart)
# For live trading: live_monitor(interval_seconds=30)
df_history = live_monitor(interval_seconds=10, n_iterations=3)

---
## Bonus: Historical Data for Backtesting

Pull historical hourly data from Open-Meteo Archive to see typical T×H ranges for London.
This helps calibrate our expectations and set better thresholds.

In [ ]:
def fetch_historical_open_meteo(start_date="2025-01-01", end_date="2025-12-31"):
    """
    Fetch historical hourly data from Open-Meteo Archive.
    Great for understanding typical ranges and building regression models.
    """
    resp = requests.get("https://archive-api.open-meteo.com/v1/archive", params={
        "latitude": LONDON_LAT, "longitude": LONDON_LON,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": "temperature_2m,relative_humidity_2m",
        "timezone": "Europe/London",
    })
    resp.raise_for_status()
    h = resp.json()["hourly"]
    
    df = pd.DataFrame({
        "time": pd.to_datetime(h["time"]),
        "temp_c": h["temperature_2m"],
        "humidity": h["relative_humidity_2m"],
    })
    df["temp_f"] = c_to_f(df["temp_c"])
    df["temp_f_rounded"] = df["temp_f"].round()
    df["t_x_h"] = df["temp_f_rounded"] * df["humidity"]
    df["hour"] = df["time"].dt.hour
    df["month"] = df["time"].dt.month
    df["dow"] = df["time"].dt.dayofweek
    return df

# Fetch 2025 data
df_hist = fetch_historical_open_meteo("2025-01-01", "2025-12-31")
print(f"Historical data: {len(df_hist)} hourly readings")
print(f"\nT×H at noon (12:00) stats:")
noon_data = df_hist[df_hist.hour == 12]
print(f"  Mean:   {noon_data['t_x_h'].mean():.0f}")
print(f"  Median: {noon_data['t_x_h'].median():.0f}")
print(f"  Std:    {noon_data['t_x_h'].std():.0f}")
print(f"  Min:    {noon_data['t_x_h'].min():.0f}")
print(f"  Max:    {noon_data['t_x_h'].max():.0f}")

# Monthly breakdown at noon
print(f"\nMonthly T×H at noon:")
monthly = noon_data.groupby("month")["t_x_h"].agg(["mean", "std"]).round(0)
print(monthly)

In [ ]:
def simulate_wx_sum_settlements(df_hist):
    """
    Simulate what WX_SUM settlement would have been for each 24h window
    (noon to noon) in the historical data.
    
    This gives us the distribution of possible settlements.
    """
    # Group by date, noon-to-noon
    df = df_hist.copy()
    # Shift so that 12:00 starts a new "day"
    df["session_date"] = (df["time"] - pd.Timedelta(hours=12)).dt.date
    
    settlements = []
    for date, group in df.groupby("session_date"):
        # Only full 24h sessions (should have ~24 hourly readings)
        if len(group) >= 23:
            # WX_SUM = sum(T_F_rounded * H / 100) but hourly not 15-min
            # Multiply by 4 to approximate 15-min intervals (each hour = 4 intervals)
            wx_sum_approx = (group["t_x_h"] / 100).sum() * 4
            
            # WX_SPOT at the end (noon)
            noon_row = group[group.hour == 12]
            if len(noon_row) > 0:
                wx_spot = noon_row.iloc[-1]["t_x_h"]
            else:
                wx_spot = None
            
            settlements.append({
                "date": date,
                "wx_sum_approx": wx_sum_approx,
                "wx_spot": wx_spot,
            })
    
    df_settlements = pd.DataFrame(settlements)
    
    print(f"Simulated {len(df_settlements)} sessions")
    print(f"\nWX_SUM distribution:")
    print(f"  Mean:   {df_settlements['wx_sum_approx'].mean():.0f}")
    print(f"  Median: {df_settlements['wx_sum_approx'].median():.0f}")
    print(f"  Std:    {df_settlements['wx_sum_approx'].std():.0f}")
    print(f"  5th %%:  {df_settlements['wx_sum_approx'].quantile(0.05):.0f}")
    print(f"  95th %%: {df_settlements['wx_sum_approx'].quantile(0.95):.0f}")
    
    print(f"\nWX_SPOT distribution:")
    spot = df_settlements['wx_spot'].dropna()
    print(f"  Mean:   {spot.mean():.0f}")
    print(f"  Std:    {spot.std():.0f}")
    print(f"  5th %%:  {spot.quantile(0.05):.0f}")
    print(f"  95th %%: {spot.quantile(0.95):.0f}")
    
    return df_settlements

df_sett = simulate_wx_sum_settlements(df_hist)